This is the final fit that I decided to go with after a lot of exploring.

In [19]:
import pandas as pd
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn import ensemble
from sklearn.utils import resample
from sklearn import svm
import numpy as np




In [20]:
#import NYC data and inspect
df = pd.read_pickle('Nydatafinal.pkl')

In [21]:
len(df)

388

In [22]:
#check imbalance
len(df[df.isbest])

20

In [23]:
df.head()

,surveyOverallRatingScore,isbest,Number_pub,HGST7FF97B36330214,HGST750A7B36330101,HGSTF40A7B36330154,HGST3A0A7B36330012,HGSTE10A7B36330204,HGST760A7B36330119,HGSTB00A7B36330236,...,HGSTA5380596310008,HGSTC6380596310025,HGST600A7B36330231,HGST11346F56140088,i10_score,totalcit,is_Rec,recent_pubs,trialcount,ArticleWords
fullname,,,,,,,,,,,,,,,,,,,,,
Sylvia Adams,10.0,False,36.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,30,2195.0,1.0,26,5,NFS1 undergoes positive selection in lung tumo...
Morton Coleman,10.0,False,90.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,77,4674.0,1.0,41,8,A Phase I Trial of High-Dose Lenalidomide and ...
Karen Haglof,9.0,False,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,1.0,1,0,Metastatic hepatocellular carcinoma in a liver...
Michael Grossbard,7.0,True,39.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,29,1440.0,0.0,17,1,Survival of Asian Females With Advanced Lung C...
Jeffrey Tepler,10.0,False,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,144.0,1.0,0,0,BiRD (Biaxin [clarithromycin]/Revlimid [lenali...


In [24]:
#normalize data

In [25]:
def normalizeseries(inputseries):
    mean = inputseries.mean()
    std = inputseries.std()
    return (inputseries-mean)/std

In [26]:
df.Number_pub = normalizeseries(df.Number_pub)
df.surveyOverallRatingScore = normalizeseries(df.surveyOverallRatingScore)
df.i10_score = normalizeseries(df.i10_score)
df.totalcit = normalizeseries(df.totalcit)
df.trialcount = normalizeseries(df.trialcount)

In [27]:
df.head()

,surveyOverallRatingScore,isbest,Number_pub,HGST7FF97B36330214,HGST750A7B36330101,HGSTF40A7B36330154,HGST3A0A7B36330012,HGSTE10A7B36330204,HGST760A7B36330119,HGSTB00A7B36330236,...,HGSTA5380596310008,HGSTC6380596310025,HGST600A7B36330231,HGST11346F56140088,i10_score,totalcit,is_Rec,recent_pubs,trialcount,ArticleWords
fullname,,,,,,,,,,,,,,,,,,,,,
Sylvia Adams,1.187003,False,0.051205,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.240777,0.077306,1.0,26,0.343482,NFS1 undergoes positive selection in lung tumo...
Morton Coleman,1.187003,False,1.064283,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.469319,0.635185,1.0,41,0.838299,A Phase I Trial of High-Dose Lenalidomide and ...
Karen Haglof,0.962887,False,-0.605419,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.543399,-0.416660,1.0,1,-0.481214,Metastatic hepatocellular carcinoma in a liver...
Michael Grossbard,0.514657,True,0.107487,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.214638,-0.092600,0.0,17,-0.316275,Survival of Asian Females With Advanced Lung C...
Jeffrey Tepler,1.187003,False,-0.605419,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.517259,-0.384254,1.0,0,-0.481214,BiRD (Biaxin [clarithromycin]/Revlimid [lenali...


In [29]:
#split into test and training
df_fit = df.drop(columns = ['ArticleWords'])
df_train, df_test = model_selection.train_test_split(df_fit, test_size=0.2,random_state = 5)

In [30]:
X_test = df_test.drop(columns = 'isbest')
y_test = df_test.isbest

In [31]:
#try 7 fold splits within training sets to fit model
kf = model_selection.KFold(n_splits=7,random_state = 4)
kf.get_n_splits(df_train)

7

In [32]:
def conf_mat(model,X,y):
    #returns confusion matrix for a model given X and y
    pred_y = model.predict(X)
    return metrics.confusion_matrix(y,pred_y)

In [33]:
def accuracy(model,X,y):
#returns accuracy score for a model given X and y
    pred_y = model.predict(X)
    return metrics.accuracy_score(y,pred_y)
    

In [34]:
#attempt a logistic regression, random forests and svm classifier
# at various points I printed some metrics of each to visualize

modellog = []
modelrf = []
modelsvm = []

for train_index, test_index in kf.split(df_train):
    df_train_i = df_train.iloc[train_index]
    df_test_i = df_train.iloc[test_index]
    df_minority = df_train_i[df_train_i.isbest==1]
    df_majority = df_train_i[df_train_i.isbest==0]
    # upsample minority to address balance
    nnew = len(df_majority)
    df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=nnew,    # to match majority class
                                 random_state=100) # reproducible results
    df_resampled = pd.concat([df_majority, df_minority_upsampled])
    y_train = df_resampled.isbest
    y_test_i = df_test_i.isbest
    X_train = df_resampled.drop('isbest',axis=1)
    X_test_i = df_test_i.drop('isbest',axis=1)
    model_log = linear_model.LogisticRegression().fit(X_train,y_train)
    modellog.append(accuracy(model_log,X_test_i,y_test_i))
    model_rfc = ensemble.RandomForestClassifier(random_state = 211).fit(X_train,y_train)
    modelrf.append(accuracy(model_rfc,X_test_i,y_test_i))
    model_svm = svm.SVC(kernel='linear', C=1).fit(X_train,y_train)
    modelsvm.append(accuracy(model_svm,X_test_i,y_test_i))

    

In [35]:
#check log reg
print(np.mean(modellog))
print(modellog)

0.851515151515
[0.8666666666666667, 0.8666666666666667, 0.79545454545454541, 0.84090909090909094, 0.90909090909090906, 0.86363636363636365, 0.81818181818181823]


In [36]:
#check random forests
print(np.mean(modelrf))
print(modelrf)

0.941847041847
[0.93333333333333335, 0.97777777777777775, 0.97727272727272729, 0.88636363636363635, 0.93181818181818177, 0.95454545454545459, 0.93181818181818177]


In [37]:
#check svm
print(np.mean(modelsvm))
print(modelsvm)

0.835209235209
[0.84444444444444444, 0.91111111111111109, 0.75, 0.81818181818181823, 0.86363636363636365, 0.86363636363636365, 0.79545454545454541]


After deliberation, it seems the average of 7 logistic regressions is the best way to go: random forests overfits. Since the "best" doctors is not necessarily 100% in our case. 

In [38]:
modellog = []
#actually fit 7 logistic regressions

for train_index, test_index in kf.split(df_train):
    df_train_i = df_train.iloc[train_index]
    df_test_i = df_train.iloc[test_index]
    df_minority = df_train_i[df_train_i.isbest==1]
    df_majority = df_train_i[df_train_i.isbest==0]
    # upsample minority
    nnew = len(df_majority)
    df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=nnew,    # to match majority class
                                 random_state=100) # reproducible results
    df_resampled = pd.concat([df_majority, df_minority_upsampled])
    y_train = df_resampled.isbest
    y_test_i = df_test_i.isbest
    X_train = df_resampled.drop('isbest',axis=1)
    X_test_i = df_test_i.drop('isbest',axis=1)
    model_log = linear_model.LogisticRegression().fit(X_train,y_train)
    modellog.append(model_log)

In [39]:
def printmodelmetrics(model,X,y):
    pred_y = model.predict(X)
    print('confusion matrix')
    print(metrics.confusion_matrix(y,pred_y))
    print('f1 score')
    print(metrics.f1_score(y,pred_y))
    print('accuracy')
    print(metrics.accuracy_score(y,pred_y))
    return metrics.f1_score(y,pred_y)

In [40]:
#check performance on test set
accuracy_test = []
for model in modellog:
    print(conf_mat(model,X_test,y_test))
    accuracy_test.append(accuracy(model,X_test,y_test))
print(np.mean(accuracy_test))
print(np.std(accuracy_test))

[[66  8]
 [ 1  3]]
[[65  9]
 [ 1  3]]
[[64 10]
 [ 1  3]]
[[65  9]
 [ 1  3]]
[[66  8]
 [ 2  2]]
[[65  9]
 [ 2  2]]
[[68  6]
 [ 1  3]]
0.875457875458
0.0163814503846


In [41]:
#This is good because we want to minimize false negatives so as to provide patients with more options
#save file
import pickle
with open('LogRegModels.pkl', "wb") as f:
   pickle.dump(modellog, f)


generate list of best doctors for total boston and ny data as well as respective confusion matrices

In [50]:
X = df.drop(columns=['isbest','ArticleTitles','ArticleWords'])

In [73]:
#calculate mean probability of 7 log regs
probability = modellog[0].predict_proba(X)
probability = [p[1] for p in probability]
for model in modellog[1:]:
    probabilities = model.predict_proba(X)
    probabilities = [p[1] for p in probabilities]
    probability = [sum(x) for x in zip(probability,probabilities)]
probability = [x/7 for x in probability]
temp = pd.DataFrame({'prob':probability},index = list(df.index))

In [55]:
df = pd.concat([df,temp],axis = 1)

In [56]:
df['isbestpred'] = False

In [57]:
for doctor in df.index:
    if df.loc[doctor,'prob']>0.5:
        df.loc[doctor,'isbestpred'] = True

In [95]:
metrics.confusion_matrix(df.isbest,df.isbestpred)

array([[318,  50],
       [  1,  19]])

### This is the confusion matrix for the entire NYC dataset. 

In [61]:
NYbest = df[df.isbestpred]
NYbest.head()

,Number_pub,surveyOverallRatingScore,is_Rec,i10_score,totalcit,HGST7FF97B36330214,HGST750A7B36330101,HGSTF40A7B36330154,HGST3A0A7B36330012,HGSTE10A7B36330204,...,HGSTC6380596310025,HGST600A7B36330231,HGST11346F56140088,isbest,recent_pubs,ArticleTitles,ArticleWords,trialcount,prob,isbestpred
fullname,,,,,,,,,,,,,,,,,,,,,
Sylvia Adams,0.051205,1.187003,1.0,0.240777,0.077306,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,False,26,NFS1 undergoes positive selection in lung tumo...,NFS1 undergoes positive selection in lung tumo...,0.343482,0.778025,True
Michael Grossbard,0.107487,0.514657,0.0,0.214638,-0.092600,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,17,Survival of Asian Females With Advanced Lung C...,Survival of Asian Females With Advanced Lung C...,-0.316275,0.617667,True
Francisco Esteva,2.096122,1.187003,1.0,2.514886,2.095706,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,False,59,A phase I trial of ganetespib in combination w...,A phase I trial of ganetespib in combination w...,0.343482,0.930175,True
Cynthia Leichman,-0.305248,1.187003,1.0,-0.229728,-0.300314,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,False,5,Phase II Study of Olaparib (AZD-2281) After St...,Phase II Study of Olaparib (AZD-2281) After St...,0.508421,0.623225,True
Daniel Cho,0.013684,1.187003,1.0,0.083942,-0.118255,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,False,18,Phase Ib Study of Safety and Pharmacokinetics ...,Phase Ib Study of Safety and Pharmacokinetics ...,0.178542,0.722588,True


In [80]:
#see how model does on Boston
df_bos = pd.read_pickle('Bostonnorm.pkl')
#remove duplicated indices
df_bos = df_bos[~df_bos.index.duplicated(keep='first')]
X_bos = df_bos.drop(columns=['isbest','ArticleTitles','ArticleWords'])
y_bos = df_bos.isbest

In [92]:
accuracy_Bos = []
for model in modellog:
    print(conf_mat(model,X_bos,y_bos))
    accuracy_Bos.append(accuracy(model,X_bos,y_bos))
print(np.mean(accuracy_Bos))
print(np.std(accuracy_Bos))


[[360  14]
 [ 31  12]]
[[357  17]
 [ 27  16]]
[[361  13]
 [ 27  16]]
[[355  19]
 [ 25  18]]
[[353  21]
 [ 26  17]]
[[351  23]
 [ 24  19]]
[[361  13]
 [ 30  13]]
0.893799246317
0.00539500368209


In [82]:
#calculate mean probability of 7 log regs in Boston
probability = modellog[0].predict_proba(X_bos)
probability = [p[1] for p in probability]
for model in modellog[1:]:
    probabilities = model.predict_proba(X_bos)
    probabilities = [p[1] for p in probabilities]
    probability = [sum(x) for x in zip(probability,probabilities)]
probability = [x/7 for x in probability]
temp = pd.DataFrame({'prob':probability},index = list(df_bos.index))
df_bos = pd.concat([df_bos,temp],axis = 1)
df_bos.head()

,Number_pub,surveyOverallRatingScore,is_Rec,i10_score,totalcit,HGST7FF97B36330214,HGST750A7B36330101,HGSTF40A7B36330154,HGST3A0A7B36330012,HGSTE10A7B36330204,...,HGSTA5380596310008,HGSTC6380596310025,HGST600A7B36330231,HGST11346F56140088,isbest,recent_pubs,ArticleTitles,ArticleWords,trialcount,prob
fullname,,,,,,,,,,,,,,,,,,,,,
Andrew Yee,-0.342770,1.187003,1.0,-0.308146,-0.379528,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,False,14,Sequencing of nontransplant treatments in mult...,Sequencing of nontransplant treatments in mult...,-0.316275,0.106066
John Clark,0.088727,1.187003,1.0,0.188499,-0.156512,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,False,14,Extensive basal cell carcinoma of the face: an...,Extensive basal cell carcinoma of the face: an...,-0.151336,0.114951
Jeffrey Clark,1.645865,1.187003,1.0,2.122798,3.248369,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,57,Phase II Study of Proton-Based Stereotactic Bo...,Phase II Study of Proton-Based Stereotactic Bo...,2.157813,0.505137
Rachel Freedman,0.351376,1.187003,1.0,0.162359,-0.266557,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,False,49,Implementation of Surgeon-Initiated Gene Expre...,Implementation of Surgeon-Initiated Gene Expre...,0.013603,0.467222
Adam Lerner,-0.061359,1.187003,1.0,0.110081,-0.150211,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,12,Anaplastic large cell lymphoma localized to th...,Anaplastic large cell lymphoma localized to th...,-0.151336,0.175808


In [87]:
df_bos['isbestpred'] = False
for doctor in df_bos.index:
    if df_bos.loc[doctor,'prob']>0.5:
        df_bos.loc[doctor,'isbestpred'] = True

In [88]:
doctor

'Anna Berkenblit'

In [89]:
Bosbest = df_bos[df_bos.isbestpred]
Bosbest.head()

,Number_pub,surveyOverallRatingScore,is_Rec,i10_score,totalcit,HGST7FF97B36330214,HGST750A7B36330101,HGSTF40A7B36330154,HGST3A0A7B36330012,HGSTE10A7B36330204,...,HGSTC6380596310025,HGST600A7B36330231,HGST11346F56140088,isbest,recent_pubs,ArticleTitles,ArticleWords,trialcount,prob,isbestpred
fullname,,,,,,,,,,,,,,,,,,,,,
Jeffrey Clark,1.645865,1.187003,1.0,2.122798,3.248369,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,57,Phase II Study of Proton-Based Stereotactic Bo...,Phase II Study of Proton-Based Stereotactic Bo...,2.157813,0.505137,True
Jacob Laubach,1.045522,1.187003,1.0,1.286344,0.341505,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,False,81,The proteasome and proteasome inhibitors in mu...,The proteasome and proteasome inhibitors in mu...,0.673360,0.758132,True
Noopur Raje,1.758429,1.187003,1.0,2.410329,1.586886,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,False,75,Myeloma bone disease: pathogenesis and treatme...,Myeloma bone disease: pathogenesis and treatme...,0.508421,0.657235,True
Andrew Evens,1.420736,1.187003,1.0,1.756850,0.440074,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,False,73,Genetic and Functional Drivers of Diffuse Larg...,Genetic and Functional Drivers of Diffuse Larg...,1.003238,0.736261,True
Jennifer Temel,1.383215,1.187003,1.0,1.364762,1.473915,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,False,84,The relationship between physical and psycholo...,The relationship between physical and psycholo...,1.662995,0.742078,True


In [90]:
len(NYbest)

69

In [91]:
len(Bosbest)

30

In [96]:
metrics.confusion_matrix(df_bos.isbest,df_bos.isbestpred)

array([[359,  15],
       [ 28,  15]])

### This is the confusion matrix for Boston data. The model was not trained or tested on these, it was used as an extrapolation.

In [97]:
NYbest.to_pickle('BestNY.pkl')
Bosbest.to_pickle('Bestbos.pkl')